In [ ]:
# Copyright 2022 Google, LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import io
import pathlib

from notebookjs import execute_js
from ruamel.yaml import RoundTripLoader

import armi
armi.configure(permissive=True)

from armi.reactor.blueprints import Blueprints
from armi.reactor.blueprints.gridBlueprint import Grids, GridBlueprint, saveToStream
from armi.utils import asciimaps
from armi.utils import textProcessors

In [ ]:
# Load an existing Blueprint file.

fName = "./testdata/anl-afci-177-blueprints.yaml"

with open(fName, "r") as bpYaml:
    bpYaml = textProcessors.resolveMarkupInclusions(
        bpYaml, root=pathlib.Path(fName).parent
    )
    bp = Blueprints.load(bpYaml, RoundTripLoader)

In [ ]:
# Visualize the reactor core layout.

print(bp.gridDesigns["core"].latticeMap)

In [ ]:
# Print out what the symbols stand for

for k, v in bp.assemDesigns.items():
    print(f"{v.specifier}: {k}")

In [ ]:
# Convert the core layout into a format that can be passed to Javascript.

asciiMapCls = asciimaps.asciiMapFromGeomAndDomain(
    geomType=bp.gridDesigns["core"].geom,
    domain=bp.gridDesigns["core"].symmetry.split()[0])

asciiMap = asciiMapCls()
asciiMap.readAscii(bp.gridDesigns["core"].latticeMap)

In [ ]:
# gridEditor.py does specify a way to generate the color palette programmatically using the
# Flags, but I am hardcoding here as a quick hack for the demo.

specifierMap = {
    "IC": {
        "color": [219, 112, 147],  # PaleVioletRed
        "name": "inner core",
    },
    "MC": {
        "color": [250, 128, 114],  # Salmon
        "name": "middle core",
    },
    "OC": {
        "color": [139, 0, 0],      # DarkRed
        "name": "outer core",
    },
    "RR": {
        "color": [195, 144, 28],   # GoldenRod
        "name": "radial reflector",
    },
    "SH": {
        "color": [128, 128, 128],  # Grey
        "name": "radial shield",
    },
    "PC": {
        "color": [0, 255, 0],      # Lime
        "name": "control",
    },
    "US": {
        "color": [0, 0, 0],        # Black
        "name": "ultimate shutdown",
    },
    "-": {
        "color": [211, 211, 211],    # Light Grey
        "name": "(nothing)",
    },
}

In [ ]:
# A simple grid, drawn using svg and d3js.

sharedData = {}

def testCallback(data):
    sharedData["points"] = data["points"]
    return True

callbacks = {
    "test": testCallback
}

with open("./dist/bundle.js", "r") as f:
    bundle = f.read()

with open("./grid_gui.css", "r") as f:
    gridGuiCss = f.read()

dataDict = {"geomtype": "hex"}
points = []
for (i, j), specifier in asciiMap.items():
    point = {
        "i": i,
        "j": j,
        "specifier": specifier,
    }
    points.append(point)
dataDict["points"] = points
dataDict["specifierMap"] = specifierMap

execute_js(
    bundle,
    main_function="bundle.main",
    callbacks=callbacks,
    data_dict=dataDict,
    css_list=[gridGuiCss],
)

In [ ]:
# Click the "Export points" button above.
# Now, this shared_data python dictionary will be populated with the edited layout,
# even though the editing logic is written in Javascript.
# Pretty cool huh?

sharedData

In [ ]:
# Now you probably want to convert the edited grid back to an ASCII map.

bp.gridDesigns["core"].gridContents = {
    (point["i"], point["j"]): point["specifier"]
    for point in sharedData["points"]
}

In [ ]:
# There! You should see the edited points below.

with io.StringIO() as f:
    saveToStream(f, bp, tryMap=True)
    print(f.getvalue())